In [341]:
import random
import numpy as np
from itertools import chain

In [1326]:
class TheGame:

    def __init__(self):
        self.drawing_pile = [i for i in range(2,100)]
        random.shuffle(self.drawing_pile) 

        self.hand = self.drawing_pile[:8]
        self.hand.sort()
        self.drawing_pile = self.drawing_pile[8:]

        self.stacks = {"up1": [1],
                       "up2": [1],
                       "down1": [100],
                       "down2": [100]}
        
        self.game_over = rf"No move possible anymore."
        self.complete = rf"Congratulations. You have beat the game."

    def recognizing_ten_diffs(self,hand):
        tenners = []
        for nr in hand:
            if type(nr) == int:
                for nr2 in hand:
                    if type(nr) == int:
                        if abs(nr - nr2) == 10:
                            diff_item = [nr, nr2]
                            diff_item.sort()
                            tenners.append(diff_item)
                    else:
                        for item in nr2:
                            if abs(item - nr) == 10:
                                diff_item = nr2.append(nr)
                                diff_item.sort()
                                tenners.append(diff_item)
                            
        ten_diffs = np.unique(np.array([np.sort(sub) for sub in tenners]), axis=0).tolist()

        ten_diffs_copy = ten_diffs.copy()
        for item in ten_diffs:
            for item2 in ten_diffs:
                if item[-1] == item2[0]:
                    new_item = list(set(item + item2))
                    new_item.sort()
                    ten_diffs_copy.append(new_item)
                    ten_diffs_copy.remove(item)
                    ten_diffs_copy.remove(item2)

        for item in ten_diffs_copy:
            hand.append(item)
            for elem in item:
                hand.remove(elem)

        return hand
    
    def ten_moves(self):
        self.hand = self.recognizing_ten_diffs(self.hand)

        for nr in self.hand:
            if type(nr) == int:
                for tenner in self.hand:
                    if type(tenner) == list:
                        if tenner[0] < nr < tenner[-1]:
                            tenner.append(nr)
                            self.hand.remove(nr)
                        tenner.sort()
                
        return self.hand
    
    def optimal_move(matrix):
        return list(zip(*np.where(matrix == matrix.min())))
    
    def analyze_move(self):
        playing_matrix = np.matrix(np.ones((4, len(self.hand))) * np.inf)

        elem_counter = 0
        for elem in self.hand:
            stack_counter = 0
            if type(elem) == int:
                for key, val in self.stacks.items():
                    if key.startswith("up"):
                        if elem > val[0]:
                            playing_matrix[stack_counter, elem_counter] = elem - val[0]

                    else:
                        if elem < val[0]:
                            playing_matrix[stack_counter, elem_counter] = val[0] - elem

                    stack_counter += 1

            else:
                for key, val in self.stacks.items():
                    if key.startswith("up"):
                        if max(elem) > val[0]:
                            playing_matrix[stack_counter, elem_counter] = min(elem) - val[0]

                    else:
                        if min(elem) < val[0]:
                            playing_matrix[stack_counter, elem_counter] = val[0] - max(elem)

                    stack_counter += 1

            elem_counter += 1

        return playing_matrix
    
    def optimal_move(self, matrix):
        return list(zip(*np.where(matrix == matrix.min())))
    
    def play_move(self):
        if len(self.hand) == 0:
            return self.complete
        
        matrix = self.analyze_move()
        print(matrix.min())
        best_move = self.optimal_move(matrix)
        stacklist = list(game_instance.stacks.keys())

        if matrix.min() == np.inf:
            return self.game_over
            
        elif len(best_move) != 1:
            tied_best_cards = []
            stacks_for_tied_best_cards = []
            dist_from_base = []

            for move in best_move:
                stack, card = list(move)[0], list(move)[1]
                tied_best_cards.append(self.hand[card])
                stacks_for_tied_best_cards.append(stack)

                if stack in [0,1]:
                    dist_from_base.append(card-1)
                else:
                    dist_from_base.append(100-card)

                choice = dist_from_base.index(min(dist_from_base))
            
            best_move = best_move[choice]
        
        try:
            stack, card = list(best_move)[0], list(best_move)[1]
        except:
            stack, card = list(best_move)[0][0], list(best_move)[0][1]
        print(rf"Best card to play: {self.hand[card]} on stack {stack + 1}.")

        card_counter = 0
        if type(self.hand[card]) == int:
            self.stacks[stacklist[stack]].insert(0, self.hand[card])
            self.hand.remove(self.hand[card])
            card_counter += 1

        else:
            if stack in [0,1]:
                while len(self.hand[card]) > 1:
                    self.stacks[stacklist[stack]].insert(0, self.hand[card][1])
                    card_counter += 1
                    self.hand[card].remove(self.hand[card][1])
                self.stacks[stacklist[stack]].insert(0, self.hand[card][0])
                card_counter += 1
            else:
                while len(self.hand[card]) > 1:
                    self.stacks[stacklist[stack]].insert(0, self.hand[card][-2])
                    card_counter += 1
                    self.hand[card].remove(self.hand[card][-2])
                self.stacks[stacklist[stack]].insert(0, self.hand[card][0])
                card_counter += 1

            self.hand.remove(self.hand[card])
        
        return card_counter, rf"New stacks:{self.stacks}, New hand: {self.hand}"
    
    def nested_list_size(self, hand):
        card_counter = 0
        for item in hand:
            if type(item) == list:
                card_counter += len(item)
            else:
                card_counter += 1
        return card_counter
        
    def draw_from_pile(self):
        if len(self.drawing_pile) > 0:
            new_card = self.drawing_pile[0]
            print(rf"Newly drawn card: {new_card}")
            self.drawing_pile.remove(new_card)
            self.hand.append(new_card)
            single_cards = [elem for elem in self.hand if isinstance(elem, int)]
            single_cards.sort()
            stacked_cards = [elem for elem in self.hand if isinstance(elem, list)]
            self.hand = single_cards + stacked_cards

    def hand_flattener(self):
        flattened_list = []
        for item in self.hand:
            if type(item) == list:
                for elem in item:
                    flattened_list.append(elem)
            else:
                flattened_list.append(item)
        sorted(flattened_list)
        return flattened_list
    
    def game_end_check(self, output):
        if output in [self.game_over, self.complete]:
            return(output)

    def full_turn(self, max_extra_move=3):
        print(rf"Starting hand and stacks: {self.hand, self.stacks}")
        if len(self.drawing_pile) >= 1:
            move = self.play_move()
            if self.game_end_check(move) != None:
                return rf"{self.game_end_check(move)} {len(self.hand) + len(self.drawing_pile)} cards left."
            if self.nested_list_size(self.hand) > 6:
                move = self.play_move()
                if self.game_end_check(move) != None:
                    return rf"{self.game_end_check(move)} {len(self.hand) + len(self.drawing_pile)} cards left."
        else:
            move = self.play_move()
            if self.game_end_check(move) != None:
                return rf"{self.game_end_check(move)} {len(self.hand) + len(self.drawing_pile)} cards left."

        while self.analyze_move().min() <= max_extra_move:
            move = self.play_move()
            self.game_end_check(move)

        self.hand = self.hand_flattener()
        for _ in range(8 - len(self.hand)):
            self.draw_from_pile()
        self.hand = self.ten_moves()

        print(rf"Current stacks: {self.stacks}")
        print(rf"Drawing pile size: {len(self.drawing_pile)}")
        print(rf"New hand: {self.hand}")

game_instance = TheGame()

In [1358]:
game_instance.full_turn()


Starting hand and stacks: ([44, 58, 63, 69, [35, 42, 45, 55]], {'up1': [96, 83, 80, 90, 89, 85, 81, 78, 77, 72, 82, 79, 71, 70, 64, 59, 56, 52, 47, 40, 31, 29, 28, 16, 11, 1], 'up2': [74, 67, 62, 50, 60, 23, 33, 17, 37, 27, 9, 5, 1], 'down1': [7, 12, 2, 6, 14, 4, 18, 19, 30, 91, 92, 95, 100], 'down2': [13, 21, 34, 24, 36, 43, 48, 38, 39, 65, 94, 99, 100]})
inf


'No move possible anymore. 34 cards left.'